# Preparation of Twitter dataset

**NOTE**: make sure, to run this notebook, you have installed [`rtree`](https://pypi.python.org/pypi/Rtree/).

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely.geometry as geoms

print gpd.__version__

0.2.0.dev-unknown


Paths:

In [2]:
lsoas_path = '../../../../data/Liverpool/shapefiles/Liverpool_lsoa11.shp'
tw_path = '../../../../data/tweets/Tweets_Liverpool.csv'

* Read point data in:

In [3]:
%%time
tw = pd.read_csv(tw_path)
# Points as geometries
pts = tw[['LON', 'LAT']].apply(geoms.Point, axis=1)
tw['geometry'] = gpd.GeoSeries(pts)
# Table as GeoDataFrame
tw = gpd.GeoDataFrame(tw)
tw.crs = {'init': u'epsg:4326'}
# Project
tw = tw.to_crs(epsg=27700)
# Add columns
xys = np.array([(pt.x, pt.y) for pt in tw['geometry']])
tw['X'] = xys[:, 0]
tw['Y'] = xys[:, 1]

CPU times: user 29.1 s, sys: 200 ms, total: 29.3 s
Wall time: 29.4 s


* Read polygons in:

In [4]:
lsoas = gpd.read_file(lsoas_path)

* Spatial join of polygons to points:

In [5]:
%%time
from geopandas.tools import sjoin

liv_tw_all = sjoin(tw, lsoas, how='left')

CPU times: user 41.5 s, sys: 40 ms, total: 41.5 s
Wall time: 41.6 s


* Keep only Liverpool tweets:

In [6]:
liv_tw = liv_tw_all.loc[liv_tw_all['LSOA11CD'].notnull(), :]\
                   .drop(['index_right', '﻿USERID', 'TWEET_TEXT'], axis=1)

* Write into a Shapefile:

In [7]:
%%time
liv_tw.to_file('../../../../data/tweets/tweets_liverpool')

CPU times: user 38.8 s, sys: 88 ms, total: 38.9 s
Wall time: 38.9 s
